In [7]:
import pandas as pd
import os

In [8]:
dir_content = [i for i in os.listdir() if not i.startswith('.')]
dir_content

['bookings.csv', 'hotels_reservations.ipynb']

In [34]:
df = pd.read_csv('bookings.csv', sep=';')
df.columns = [j.lower() for j in df.columns]
df.tail()

# Hotel – тип отеля (City Hotel или Resort Hotel)  
# Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
# Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
# Arrival full date – полная дата прибытия
# Arrival date year – год прибытия  
# Arrival date month – месяц прибытия  
# Arrival date week number – номер недели прибытия
# Arrival date day of month – день прибытия
# Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
# Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
# Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
# Adults – число взрослых
# Children – число детей
# Babies – число младенцев 
# Meal – выбранный тип питания
# Country – страна происхождения клиента
# Reserved room type – тип зарезервированного номера
# Assigned room type – тип полученного номера (может отличаться от забронированного)
# Customer type – тип бронирования
# Reservation status – значение последнего статуса брони: Canceled - было отменено клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не зарегистрировался и сообщил администрации отеля причину
# Reservation status date – дата обновления статуса

,hotel,is canceled,lead time,arrival full date,arrival date year,arrival date month,arrival date week number,arrival date day of month,stays in weekend nights,stays in week nights,...,adults,children,babies,meal,country,reserved room type,assigned room type,customer type,reservation status,reservation status_date
119385,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,2,0.0,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06
119386,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,3,0.0,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07
119387,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07
119388,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07
119389,City Hotel,0,205,2017-08-29,2017,August,35,29,2,7,...,2,0.0,0,HB,DEU,A,A,Transient,Check-Out,2017-09-07


In [53]:
all_reservings = df.shape[0]
df = df.rename(columns={"is canceled": "is_canceled", 
                       "stays total nights": "stays_total_nights",
                       "reserved room type": "reserved_room_type", 
                       "assigned room type": "assigned_room_type",
                       "arrival date month": "arrival_date_month",
                       "arrival date year": "arrival_date_year",
                       "arrival full date": "arrival_full_date"})

df.arrival_full_date = pd.to_datetime(df["arrival_full_date"])
if all_reservings == df.shape[0]:
    print("The dataframe change was successful")

The dataframe change was successful


In [27]:
# Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.
countries_with_most_successful_reservings = df.query("is_canceled == 0") \
                                              .country \
                                              .value_counts()

countries_with_most_successful_reservings

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
       ...  
BHR        1
DJI        1
MLI        1
NPL        1
FRO        1
Name: count, Length: 165, dtype: int64

In [30]:
# На сколько ночей в среднем бронируют отели разных типов?

average_night_for_hotel_types = df.groupby("hotel", as_index=False).agg({'stays_total_nights':'mean'}).round(2)
average_night_for_hotel_types

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [33]:
# Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). 
# Сколько подобных наблюдений встретилось?

count_difference_beetwen_reserved_assigned_room_type = df.query("reserved_room_type != assigned_room_type").shape[0]
count_difference_beetwen_reserved_assigned_room_type

14917

In [76]:
# Проанализируйте даты запланированного прибытия. 
# – На какой месяц чаще всего успешно(т.е. бронирования не отменялись) оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?
# – Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов

count_success_bookings_per_month = df.query("is_canceled == 0"). \
                            groupby(["arrival_date_month", "arrival_date_year"], as_index=False). \
                            agg({"is_canceled": "count"}). \
                            sort_values("arrival_date_year", ascending=False)


most_popular_month_2016 = count_success_bookings_per_month.query("arrival_date_year == 2016").iloc[0]["arrival_date_month"]
most_popular_month_2017 = count_success_bookings_per_month.query("arrival_date_year == 2017").iloc[0]["arrival_date_month"]

if most_popular_month_2016 != most_popular_month_2017:
    print(f"Most popular month was changed: 2016 - {most_popular_month_2016}, 2017 - {most_popular_month_2017}")
else:
    print(f"Most popular month has not changed in relation to 2017: {most_popular_month_2017}")

Most popular month was changed: 2016 - October, 2017 - July


In [90]:
# Создайте колонку total_kids, объединив children и babies. Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?
df["total_kids"] = df["children"] + df["babies"]
if all_reservings == df.shape[0]:
    print("The dataframe change was successful")

mean_reservings_with_kids_by_hotel_types = df.groupby("hotel", as_index=False).agg({"total_kids": "mean"}).round(2).max()
mean_reservings_with_kids_by_hotel_types

The dataframe change was successful


hotel         Resort Hotel
total_kids            0.14
dtype: object

In [104]:
# Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids),
# в противном случае – False. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate).
# Укажите, среди какой группы показатель выше.

df["has_kids"] = df["total_kids"] > 0
if all_reservings == df.shape[0]:
    print("The dataframe change was successful")

churn_rate_with_kids = df.query("is_canceled == 1 and has_kids == True").shape[0] / df.query("has_kids == True").shape[0]
churn_rate_with_kids = round(churn_rate_with_kids * 100, 2)
churn_rate_with_kids

The dataframe change was successful


34.92